# 16. 基于MindSpore设置数据集Batch

本小节主要介绍如何在mindspore中设置batch。

# 1.实验目的
- 理解batch-size的作用；
- 掌握在MindSpore框架设置batch-size；
- 了解在机器学习中batch-size相关函数的使用方法。

# 2.batch-size知识点介绍
- batch操作将数据集分批，分别输入到训练系统中进行训练，可以减少训练轮次，达到加速训练过程的目的。
- batch_size：表示单次传递给用以训练的数据（样本）个数。例如当前训练集有1000个数据。如果设置batch_size=100，那么程序首先会用数据集中的前100个参数，即第1-100个数据来训练模型。当训练完成后更新权重，再使用第101-200的个数据训练，直至第十次使用完训练集中的1000个数据后停止。

# 3 实验环境
在动手进行实践之前，需要注意以下几点：
* 确保实验环境正确安装，包括安装MindSpore。安装过程：首先登录[MindSpore官网安装页面](https://www.mindspore.cn/install)，根据安装指南下载安装包及查询相关文档。同时，官网环境安装也可以按下表说明找到对应环境搭建文档链接，根据环境搭建手册配置对应的实验环境。
* 推荐使用交互式的计算环境Jupyter Notebook，其交互性强，易于可视化，适合频繁修改的数据分析实验环境。
* 实验也可以在华为云一站式的AI开发平台ModelArts上完成。
* 推荐实验环境：MindSpore版本=MindSpore 2.0；Python环境=3.7


|  硬件平台 |  操作系统  | 软件环境 | 开发环境 | 环境搭建链接 |
| :-----:| :----: | :----: |:----:   |:----:   |
| CPU | Windows-x64 | MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.1节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| GPU CUDA 10.1|Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.2节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| Ascend 910  | Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第四章](./MindSpore环境搭建实验手册.docx)|

# 4. 数据处理
本实验通过样例先构建了一个数据集，然后分别展示了丢弃多余数据与否的数据集分批结果，其中批大小为2。

## 4.1 数据准备

MNIST数据集是机器学习领域中经典的数据集，由60,000个训练样本和10,000个测试样本组成，每张图片都是28x28像素大小的灰度图像，数字从0到9。

MNIST数据集主要用于图像识别领域的研究，特别是用于机器学习算法的测试和比较。由于它的简单性和易于使用，MNIST已经成为机器学习社区中的标准数据集之一。

以下是几个示例图像：

![jupyter](./fig/Fig001.png)

每个图像都被转换成784个数字的一维向量（28×28=784）。这些数字表示像素的灰度值，值的范围从0（黑色）到255（白色）。

## 4.2 数据下载
从开放数据集中下载MNIST数据集的压缩包,并解压储存在项目的根目录下。

In [1]:
# 从开放数据集中下载MNIST数据集
from download import download

url = "https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/" \
      "notebook/datasets/MNIST_Data.zip"
path = download(url, "./", kind="zip", replace=True)


file_sizes: 100%|██████████████████████████| 10.8M/10.8M [00:00<00:00, 15.5MB/s]
Extracting zip file...
Successfully downloaded / unzipped to ./


下载后的数据集目录结构如下：



/MNIST_Data\
├── train\
&emsp;├──train-images-idx3-ubyte\
&emsp;└──train-labels-idx1-ubyte\
├──text\
&emsp;├──t10k-images-idx3-ubyte\
&emsp;└──t10k-labels-idx1-ubyte

## 4.3 设置超参数

集中设置在模型训练过程中需要使用到的超参数的具体数值，方便在后续调试代码时修改超参数。

In [2]:
BATCH_SIZE= 64       # batch的大小
LEARNING_RATE = 1e-2 # 学习率
EPOCH = 3            # 迭代次数

## 4.4 获取数据集对象
利用mindspore中的dataset相关的函数读取数据集中的数据。

In [3]:
from mindspore.dataset import transforms
# 读取和解析Manifest数据文件构建数据集
from mindspore.dataset import MnistDataset
train_dataset = MnistDataset('MNIST_Data/train')
test_dataset = MnistDataset('MNIST_Data/test')

## 4.5 数据处理 
MindSpore的dataset使用数据处理流水线（Data Processing Pipeline），需指定map、batch、shuffle等操作。这里我们使用map对图像数据及标签进行变换处理，然后将处理好的数据集打包为大小为64的batch。

In [4]:
# MindSpore库
import mindspore
# 神经网络模块
from mindspore import nn
# 常见算子操作
from mindspore import ops
# 图像增强模块
from mindspore.dataset import vision
def datapipe(dataset, batch_size):
    image_transforms = [
        # 基于给定的缩放和平移因子调整图像的像素大小。输出图像的像素大小为：output = image * rescale + shift。
        # 此处rescale取1.0 / 255.0，shift取0
        vision.Rescale(1.0 / 255.0, 0),
        # 正则化 均值为0.1307，标准差为0.3081（查自官网）
        vision.Normalize(mean=(0.1307,), std=(0.3081,)),
        # 将输入图像的shape从 <H, W, C> 转换为 <C, H, W>
        vision.HWC2CHW()
    ]
    # 将输入的Tensor转换为指定的数据类型。
    label_transform = transforms.TypeCast(mindspore.int32)

    # map给定一组数据增强列表，按顺序将数据增强作用在数据集对象上。
    dataset = dataset.map(image_transforms, 'image')
    dataset = dataset.map(label_transform, 'label')
    # 将数据集中连续 batch_size 条数据组合为一个批数据
    dataset = dataset.batch(batch_size)
    return dataset
# 对数据集进行transfrom和batch
train_dataset = datapipe(train_dataset, BATCH_SIZE)
test_dataset = datapipe(test_dataset, BATCH_SIZE)

# 5. 实验过程

模型的构建包括如下部分：
- 导入库和函数；
- 模型构建；

## 5.1 导入所需库和函数
在代码最开始集中导入整个实验中所需要使用的库和函数。

In [5]:
# MindSpore库
import mindspore
# 神经网络模块
from mindspore import nn
# 常见算子操作
from mindspore import ops
# 图像增强模块
from mindspore.dataset import vision
# 通用数据增强
from mindspore.dataset import transforms
# 读取和解析Manifest数据文件构建数据集
from mindspore.dataset import MnistDataset

## 5.2 模型构建
使用mindspore中提供的相关函数构建训练使用的神经网络模型，包括平图层，致密连接层等，并在construct中完成神经网络的前向构造。

In [6]:
# 定义模型
# MindSpore 中提供用户通过继承 nn.Cell 来方便用户创建和执行自己的网络
class Network(nn.Cell): 
    # 自定义的网络中，需要在__init__构造函数中申明各个层的定义
    def __init__(self): 
         # 继承父类nn.cell的__init__方法
        super().__init__()         
        # nn.Flatten为输入展成平图层，即去掉那些空的维度
        self.flatten = nn.Flatten()
        # 使用SequentialCell对网络进行管理
        self.dense_relu_sequential = nn.SequentialCell(
            # nn.Dense为致密连接层，它的第一个参数为输入层的维度，第二个参数为输出的维度，
            # 第三个参数为神经网络可训练参数W权重矩阵的初始化方式，默认为normal
            # nn.ReLU()非线性激活函数，它往往比论文中的sigmoid激活函数具有更好的效益
            nn.Dense(28 * 28, 512), # 致密连接层 输入28*28 输出512
            nn.ReLU(),              # ReLU层
            nn.Dense(512, 512),     # 致密连接层 输入512 输出512
            nn.ReLU(),              # ReLu层
            nn.Dense(512, 10)       # 致密连接层 输入512 输出10
        )
    # 在construct中实现层之间的连接关系，完成神经网络的前向构造
    def construct(self, x):
         #调用init中定义的self.flatten()方法 
        x = self.flatten(x)
        #调用init中的self.dense_relu_sequential()方法
        logits = self.dense_relu_sequential(x)
        # 返回模型
        return logits
model = Network()
print(model)

Network<
  (flatten): Flatten<>
  (dense_relu_sequential): SequentialCell<
    (0): Dense<input_channels=784, output_channels=512, has_bias=True>
    (1): ReLU<>
    (2): Dense<input_channels=512, output_channels=512, has_bias=True>
    (3): ReLU<>
    (4): Dense<input_channels=512, output_channels=10, has_bias=True>
    >
  >


# 6、模型训练

在模型训练中，一个完整的训练过程（step）需要实现以下三步：
1. 正向计算：模型预测结果（logits），并与正确标签（label）求预测损失（loss）。
2. 反向传播：利用自动微分机制，自动求模型参数（parameters）对于loss的梯度（gradients）。
3. 参数优化：将梯度更新到参数上。

MindSpore使用函数式自动微分机制，因此针对上述步骤需要实现：
1. 正向计算函数定义。
2. 通过函数变换获得梯度计算函数。
3. 训练函数定义，执行正向计算、反向传播和参数优化。

In [7]:
# 实例化损失函数和优化器
# 计算预测值和目标值之间的交叉熵损失
loss_fn = nn.CrossEntropyLoss() 
#构建一个Optimizer对象，能够保持当前参数状态并基于计算得到的梯度进行参数更新 此处使用随机梯度下降算法
optimizer = nn.SGD(model.trainable_params(), learning_rate=LEARNING_RATE) 

In [8]:
def train(model, dataset, loss_fn, optimizer):
    # 定义 forward 函数
    def forward_fn(data, label):
        # 将数据载入模型
        logits = model(data)
        # 根据模型训练获取损失函数值
        loss = loss_fn(logits, label)
        return loss, logits
    # 调用梯度函数，value_and_grad()为生成求导函数，用于计算给定函数的正向计算结果和梯度
    grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)
    # 定义一步训练的函数
    def train_step(data, label):
        # 计算梯度，记录变量是怎么来的
        (loss, _), grads = grad_fn(data, label)
        # 获得损失 depend用来处理操作间的依赖关系
        loss = ops.depend(loss, optimizer(grads))
        return loss
    size = dataset.get_dataset_size()
    model.set_train()
    for batch, (data, label) in enumerate(dataset.create_tuple_iterator()):
        # 批量训练获得损失值
        loss = train_step(data, label)
        # 当完成所有数据样本的训练
        if batch % 100 == 0:
            loss, current = loss.asnumpy(), batch
            print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")

定义测试函数，通过调用mindspore中的相关函数，使用accuracy以及平均损失等指标评估预测模型的性能。

In [9]:
def test(model, dataset, loss_fn):
    num_batches = dataset.get_dataset_size()
    model.set_train(False)
    total, test_loss, correct = 0, 0, 0
    for data, label in dataset.create_tuple_iterator(): # 遍历所有测试样本数据
        pred = model(data)                              # 根据已训练模型获取预测值
        total += len(data)                              # 统计样本数
        test_loss += loss_fn(pred, label).asnumpy()     # 统计样本损失值
        correct += (pred.argmax(1) == label).asnumpy().sum()# 统计预测正确的样本个数
    test_loss /= num_batches                              # 求得平均损失
    correct /= total                                      # 计算accuracy
    print(f"Test: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

训练过程需多次迭代数据集，一次完整的迭代称为一轮（epoch）。在每一轮，遍历训练集进行训练，结束后使用测试集进行预测。打印每一轮的loss值和预测准确率（Accuracy），可以看到loss在不断下降，Accuracy在不断提高。

In [10]:
for t in range(EPOCH):
    print(f"Epoch {t+1}\n-------------------------------")
    train(model, train_dataset, loss_fn, optimizer)      # 训练模型
    test(model, test_dataset, loss_fn)                   # 测试模型
print("finished!")

Epoch 1
-------------------------------
loss: 2.302351  [  0/938]
loss: 2.285768  [100/938]
loss: 2.262563  [200/938]
loss: 2.172953  [300/938]
loss: 1.970719  [400/938]
loss: 1.339656  [500/938]
loss: 0.907637  [600/938]
loss: 0.852889  [700/938]
loss: 0.754258  [800/938]
loss: 0.475010  [900/938]
Test: 
 Accuracy: 85.1%, Avg loss: 0.526761 

Epoch 2
-------------------------------
loss: 0.590947  [  0/938]
loss: 0.616612  [100/938]
loss: 0.469197  [200/938]
loss: 0.384189  [300/938]
loss: 0.387355  [400/938]
loss: 0.316738  [500/938]
loss: 0.460525  [600/938]
loss: 0.417275  [700/938]
loss: 0.334434  [800/938]
loss: 0.409174  [900/938]
Test: 
 Accuracy: 89.9%, Avg loss: 0.339449 

Epoch 3
-------------------------------
loss: 0.306898  [  0/938]
loss: 0.437584  [100/938]
loss: 0.307959  [200/938]
loss: 0.183329  [300/938]
loss: 0.296107  [400/938]
loss: 0.509621  [500/938]
loss: 0.459413  [600/938]
loss: 0.207709  [700/938]
loss: 0.476348  [800/938]
loss: 0.113756  [900/938]
Test: 
 

# 7、模型预测

模型训练完成后，将训练好的模型保存至指定的路径，方便后续的实验中在使用该模型的时候能够加载模型。\
实例化一个随机初始化的模型，并将训练好的超参数加载到刚才初始化的模型中。\
使用测试数据测试模型的识别能力，并打印识别出的数字。

In [11]:
# 保存checkpoint时的配置策略
mindspore.save_checkpoint(model, "model.ckpt")
print("Saved Model to model.ckpt")

Saved Model to model.ckpt


In [12]:
# 实例化一个随机初始化的模型 
model = Network()
# 加载检查点，加载参数到模型 
param_dict = mindspore.load_checkpoint("model.ckpt")
param_not_load = mindspore.load_param_into_net(model, param_dict)

[]


In [13]:
model.set_train(False)
for data, label in test_dataset:
    pred = model(data)
    predicted = pred.argmax(1)
    print(f'Predicted: "{predicted[:10]}", Actual: "{label[:10]}"')
    break

Predicted: "[0 4 7 2 5 3 9 9 6 8]", Actual: "[0 4 7 2 5 3 9 4 6 8]"
